# requests와 BeautifulSoup으로 웹 크롤러 만들기

## 웹 크롤러란?

우리가 어떤 정보를 브라우저에서만 보는 것 뿐 아니라 그 정보들을 내가 이용하기 편한 방식(ex: json)으로 로컬에 저장하고 싶을 때가 있다.  
웹 사이트에서 원하는 정보만을 원하는 방식으로 가공하여 얻는 것이다.

## 웹에서 정보 가져오기

## Requests

python에 있는 유명한 http request 라이브러리이다.

설치 : ```pip install requests```

In [3]:
import requests

# HTTP GET Request
req = requests.get("https://github.com/wansiklee")


# HTML 소스 가져오기
html = req.text

# HTTP header 가져오기
header = req.headers

# HTTP Status 가져오기 (200: 정상)
status = req.status_code

# HTTP가 정상적으로 되었는지 (True/False)
is_ok = req.ok

위에서 사용할 것은 HTML 소스를 이용하는 것이다. 따라서 ```html=req.text```를 이용한다.

## BeautifulSoup

Requests는 html을 '의미있는', 즉 Python이 이해하는 객체 구조로 만들어주지는 못한다. 위에서 req.text는 python의 문자열(str)객체를 반환할 뿐이기 때문에 정보를 추출하기가 어렵다.

따라서 ```BeautifulSoup```을 이용하게 된다. 이 BeautifulSoup은 html 코드를 Python이 이해하는 객체 구조로 변환하는 Parsing을 맡고 있고, 이 라이브러리는 이용해 우리는 제대로 된 '의미있는' 정보를 추출해 낼 수 있다.

설치 : ```pip install bs4```

In [4]:
import requests
from bs4 import BeautifulSoup

# HTTP GET Requests
req = requests.get("https://github.com/wansiklee")

# HTML 소스 가져오기
html = req.text

# BeautifulSoup으로 html 소스를 python 객체로 변환하기
# 첫 번째 인자는 html 소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시
# 여기서는 Python 내장 html.parser를 이용함
soup = BeautifulSoup(html, 'html.parser')

이제 ```soup``` 객체에서 원하는 정보를 찾아낼 수 있다.

BeautifulSoup에서는 여러가지 기능을 제공하는데, 여기서는 ```select```를 이용한다.  
```select```는 CSS Selector를 이용해 조건과 일치하는 모든 객체들을 List로 반환해준다.

예시로 내 깃허브 메인 페이지의 repository 제목들을 가져와 본다.
![title](gitreptitle.png)

크롬에 내장된 검사도구(요소 위에서 우측 클릭 후 검사)를 이용해보면 현재 title은 span 태그로 구성되어있는 것을 알 수 있다.  
보다 정확하게 가져오기 위해 CSS Selector를 확인해 본다. (해당 태그 위의 우측 클릭 후 copy -> copy selector)

아래와 같은 코드가 나왔다.  
```#user-45587474-pinned-items-reorder-form > ol > li:nth-child(1) > div > div > div > a > span```

하지만 ```nth-child(1)``` 등이 붙어있는 것으로 보아 현재 요소를 '정확하게' 특정하고 있기 때문에, 좀 더 유연하게 만들어 주기 위해 아래와 같이 바꿔준다.(위 코드는 단 하나의 링크만을 특정하고, 아래 코드는 css selector에 일치하는 모든 요소를 가리킨다.)

```ol > li > div > div > div > a > span```

In [19]:
import requests
from bs4 import BeautifulSoup

req = requests.get("https://github.com/wansiklee")
html = req.text
soup = BeautifulSoup(html, 'html.parser')

# CSS Selector를 통해 html요소들을 찾아낸다.
my_titles = soup.select( 'ol > li > div > div > div > a > span')

위 코드에서 my_titles는 string의 list가 아니라 soup객체들의 list이다. 따라서 태그의 속성들도 이용할 수 있는데, a태그의 경우 href 속성이 대표적인 예시다.

soup객체는 <태그><태그/>로 구성된 요소를 Python이 이해하는 상태로 바꾼 것이라고 볼 수 있다. 따라서 여러가지 조작이 가능하다.

In [33]:
import requests
from bs4 import BeautifulSoup

req = requests.get("https://github.com/wansiklee/")
html = req.text
soup = BeautifulSoup(html, 'html.parser')
my_titles = soup.select( 'ol > li > div > div > div > a > span')

# my_titles는 list 객체
for title in my_titles:
    # tag안의 텍스트
    print(title.text)
    # tag의 속성을 가져오기(ex: href, title속성...)
    print(title.get('title'))

Chatbot_seq2seq
Chatbot_seq2seq
Sentiment-Analysis
Sentiment-Analysis
Text-Similarity
Text-Similarity
DeepLearning_with_python
DeepLearning_with_python
myMomentum
myMomentum
PaintJS
PaintJS


위와 같이 코드를 처리할 경우 span 태그 안의 텍스트와 span 태그 안의 title 속성 값을 가져오게 된다. 위 코드에서 title 객체는 python의 dictionary와 같이 태그의 속성들을 저장한다. ```title.get('속성이름')``` 이나 ```title['속성이름']```처럼 이용할 수 있다.

```select```를 통해 요소들을 가져온 이후에는 각자가 생각하는 방식으로 python코드를 이용해 저장하면 된다.

### 정리 예제

아래 코드는 크롤링한 데이터를 만든 python 파일과 같은 위치에 ```result.json```을 만들어 저장하는 예제이다.

In [ ]:
# 만든 파이썬 파일 xxx.py
import requests
from bs4 import BeautifulSoup
import json
import os

# python파일의 위치
BASE_DIR = os.path.dirname(os.path.abspath(__file__))

req = requests.get("https://github.com/wansiklee/")
html = req.text
soup = BeautifulSoup(html, 'html.parser')
my_titles = soup.select( 'ol > li > div > div > div > a > span')

data = {}

for title in my_titles:
    data[title.text] = title.get('title')

with open(os.path.join(BASE_DIR, 'result.json'), 'w+') as json_file:
    json.dump(data, json_file)